In [9]:
import numpy as np #importa a biblioteca usada para trabalhar com vetores de matrizes
%run normalization_utils.py

X, Y, feature_names = build_bow()

Pegando notícias verdadeiras...
Pegando notícias falsas...
Quantidade de notícias verdadeiras: 3600
Quantidade de notícias falsas: 3600
Montando o BOW...
Quantidade de features: 44685
10 primeiros valores de Y: [0 0 0 0 0 0 0 0 0 0]
10 ultimos valores de Y: [1 1 1 1 1 1 1 1 1 1]


In [10]:
# Testando metodo para converter uma string numa amostra para predição
sentence = 'Tinha uma pedra no meio do caminho, no meio do caminho tinha uma pedra. Qual é o sentido da vida se a nossa existência é tão efemera.'

a = sentence_to_features(sentence, feature_names)

print(a[a!=0])
print(len(a))

[2 1 2 2 1 1 1]
44685


In [16]:
print('Calculando folds...')
folds = stratified_kfolds(Y, 5)
print('Folds calculados')

Calculando folds...
Folds calculados
